# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/victormoncada/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/victormoncada/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [5]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/victormoncada/Documents/Clickit/maven/The-AI-Engineer-Challenge/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/victormoncada/Documents/Clickit/maven/The-AI-Engineer-Challenge/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/victormoncada/Documents/Clickit/maven/The-AI-Engineer-Challenge/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [6]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [7]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [8]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '96b4b0'. Skipping!
Property 'summary' already exists in node '0775ed'. Skipping!
Property 'summary' already exists in node '996c3a'. Skipping!
Property 'summary' already exists in node '06ba9e'. Skipping!
Property 'summary' already exists in node '93875a'. Skipping!
Property 'summary' already exists in node 'd694be'. Skipping!
Property 'summary' already exists in node 'e7dc71'. Skipping!
Property 'summary' already exists in node 'acf55d'. Skipping!
Property 'summary' already exists in node 'e611f2'. Skipping!
Property 'summary' already exists in node '72caeb'. Skipping!
Property 'summary' already exists in node '8fbd89'. Skipping!
Property 'summary' already exists in node '6c28f4'. Skipping!
Property 'summary' already exists in node '3c67c2'. Skipping!
Property 'summary' already exists in node 'e1355e'. Skipping!
Property 'summary' already exists in node '0668d7'. Skipping!
Property 'summary' already exists in node '4980c6'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '96b4b0'. Skipping!
Property 'summary_embedding' already exists in node 'e7dc71'. Skipping!
Property 'summary_embedding' already exists in node 'e1355e'. Skipping!
Property 'summary_embedding' already exists in node '06ba9e'. Skipping!
Property 'summary_embedding' already exists in node 'acf55d'. Skipping!
Property 'summary_embedding' already exists in node '0775ed'. Skipping!
Property 'summary_embedding' already exists in node '8fbd89'. Skipping!
Property 'summary_embedding' already exists in node '0668d7'. Skipping!
Property 'summary_embedding' already exists in node 'a15696'. Skipping!
Property 'summary_embedding' already exists in node '72caeb'. Skipping!
Property 'summary_embedding' already exists in node '3c67c2'. Skipping!
Property 'summary_embedding' already exists in node '996c3a'. Skipping!
Property 'summary_embedding' already exists in node '93875a'. Skipping!
Property 'summary_embedding' already exists in node 'd694be'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 86, relationships: 712)

We can save and load our knowledge graphs as follows.

In [9]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 712)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [10]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [11]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

#### ANSWER #1:

1. SingleHopSpecificQuerySynthesizer: Generates simple questions that need only ONE piece of information from ONE source. Example: "What is the product price?"

2. MultiHopAbstractQuerySynthesizer: Generates complex questions requiring MULTIPLE sources and asks for HIGH-LEVEL understanding or summaries. Example: "What are the main themes across all documents?"

3. MultiHopSpecificQuerySynthesizer: Generates complex questions requiring MULTIPLE sources but asks for SPECIFIC FACTS that need to be combined. Example: "What's the revenue difference between Q1 and Q3?"

Finally, we can use our `TestSetGenerator` to generate our testset!

In [12]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/10 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Whn was ChatGPT launced in Novembar 2022 and h...,[Introduction ChatGPT launched in November 202...,Introduction ChatGPT launched in November 2022...,single_hop_specifc_query_synthesizer
1,How is ChatGPT used for work-related purposes ...,[Table 1: ChatGPT daily message counts (millio...,Nearly 80% of all ChatGPT usage falls into thr...,single_hop_specifc_query_synthesizer
2,How is ChatGPT usage distributed across differ...,[Variation by Occupation Figure 23 presents va...,Variation by occupation shows that users in hi...,single_hop_specifc_query_synthesizer
3,Personal Reflection how ChatGPT impact society?,[Conclusion This paper studies the rapid growt...,The context discusses the rapid growth and usa...,single_hop_specifc_query_synthesizer
4,how chatgpt use in different jobs and its big ...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,the first part <1-hop> explains variation in c...,multi_hop_abstract_query_synthesizer
5,How do the growth in message volume for work a...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The data shows that the total message volume f...,multi_hop_abstract_query_synthesizer
6,How do the conversation topics and user occupa...,[<1-hop>\n\nVariation by Occupation Figure 23 ...,The context indicates that users in management...,multi_hop_abstract_query_synthesizer
7,how openai chatgpt use grow fast and impact so...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,"The context shows that ChatGPT, developed by O...",multi_hop_specific_query_synthesizer
8,US ChatGPT use in US how much in US,[<1-hop>\n\nConclusion This paper studies the ...,The context shows that ChatGPT usage in the US...,multi_hop_specific_query_synthesizer
9,how much chatgpt used in july 2025 and what to...,[<1-hop>\n\nConclusion This paper studies the ...,"By July 2025, ChatGPT was used weekly by more ...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [13]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '8c7dc3'. Skipping!
Property 'summary' already exists in node '3fd155'. Skipping!
Property 'summary' already exists in node 'd65145'. Skipping!
Property 'summary' already exists in node '68ee78'. Skipping!
Property 'summary' already exists in node '71f0d1'. Skipping!
Property 'summary' already exists in node 'e0c773'. Skipping!
Property 'summary' already exists in node '3ae3d5'. Skipping!
Property 'summary' already exists in node 'e51f9b'. Skipping!
Property 'summary' already exists in node '243230'. Skipping!
Property 'summary' already exists in node 'f0a4d8'. Skipping!
Property 'summary' already exists in node '0dcadb'. Skipping!
Property 'summary' already exists in node '297cfd'. Skipping!
Property 'summary' already exists in node 'b184a8'. Skipping!
Property 'summary' already exists in node 'e1f75f'. Skipping!
Property 'summary' already exists in node '1b0858'. Skipping!
Property 'summary' already exists in node 'bd7b4d'. Skipping!
Property

...

Property 'summary_embedding' already exists in node '243230'. Skipping!
Property 'summary_embedding' already exists in node '3fd155'. Skipping!
Property 'summary_embedding' already exists in node '8c7dc3'. Skipping!
Property 'summary_embedding' already exists in node 'd65145'. Skipping!
Property 'summary_embedding' already exists in node 'e0c773'. Skipping!
Property 'summary_embedding' already exists in node '68ee78'. Skipping!
Property 'summary_embedding' already exists in node 'e51f9b'. Skipping!
Property 'summary_embedding' already exists in node 'c55797'. Skipping!
Property 'summary_embedding' already exists in node '0dcadb'. Skipping!
Property 'summary_embedding' already exists in node 'b184a8'. Skipping!
Property 'summary_embedding' already exists in node '3ae3d5'. Skipping!
Property 'summary_embedding' already exists in node '71f0d1'. Skipping!
Property 'summary_embedding' already exists in node 'f0a4d8'. Skipping!
Property 'summary_embedding' already exists in node '297cfd'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [16]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Korinek and Suh 2024 what they say about AI?,[Introduction ChatGPT launched in November 202...,"The context mentions Korinek and Suh, 2024, in...",single_hop_specifc_query_synthesizer
1,How is ChatGPT used in practical guidance acco...,[Table 1: ChatGPT daily message counts (millio...,Nearly 80% of all ChatGPT usage falls into thr...,single_hop_specifc_query_synthesizer
2,Other Professional what does it mean in ChatGP...,[Variation by Occupation Figure 23 presents va...,Variation by Occupation Figure 23 reports resu...,single_hop_specifc_query_synthesizer
3,How does Writing relate to the overall use of ...,[Conclusion This paper studies the rapid growt...,"Writing is by far the most common work use, ac...",single_hop_specifc_query_synthesizer
4,How do the large language models (LLMs) like C...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"ChatGPT, launched in November 2022 and based o...",multi_hop_abstract_query_synthesizer
5,How does the message volume comparison between...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,"The context shows that in June 2024, total mes...",multi_hop_abstract_query_synthesizer
6,How does the increase in non-work message data...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,The data shows that non-work messages increase...,multi_hop_abstract_query_synthesizer
7,How does the increase in non-work mesages (M) ...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,The data shows that non-work messages increase...,multi_hop_abstract_query_synthesizer
8,how do 700 million users use chatgpt for data ...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The provided context does not explicitly detai...,multi_hop_specific_query_synthesizer
9,"How do the 700 million users of ChatGPT, as of...",[<1-hop>\n\nIntroduction ChatGPT launched in N...,"By July 2025, ChatGPT had been used weekly by ...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [14]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [15]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [16]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [18]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [19]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [20]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [21]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [22]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [23]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [24]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'Based on the provided context, people are using AI, particularly generative AI like ChatGPT, in a variety of ways both at work and outside of work. At work, AI is performing workplace tasks by either augmenting or automating human labor. Users leverage generative AI for producing writing, software code, spreadsheets, and other digital products, distinguishing it from traditional web search engines. People also use AI with different intents, including asking for information or advice, doing tasks, and expressing themselves. Moreover, AI agents can act as co-workers producing output or as co-pilots providing advice and boosting human problem-solving productivity. Additionally, use cases include personal self-expression such as relationships, personal reflection, games, and role play, though these make up smaller portions of usage. Overall, generative AI is flexible and widely applied for enhancing productivity, creativity, and information-seeking.\n\nIn summary: People are using AI to a

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [25]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [27]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:  Accuracy 
- `labeled_helpfulness_evaluator`:Utility
- `dopeness_evaluator`: Quality of Experience

## LangSmith Evaluation

In [28]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'only-wave-72' at:
https://smith.langchain.com/o/9caeceb1-c4f0-4b46-af6b-bedb25ad4d93/datasets/949c36f7-81d7-463b-91d9-a21245747a40/compare?selectedSessions=1144cb19-410d-4a46-95ad-6912e84ec793




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"US ChatGPT use mainly for work or non work, ho...",Based on the provided context:\n\n- ChatGPT us...,None,"In the US, about 70% of ChatGPT consumer queri...",1,1,0,4.734578,3ae81311-9dd5-4446-a1eb-71d9e8d36649,f38b1935-3169-43c1-b96d-116de7870dd9
1,How does the growth of ChatGPT's usage in the ...,The growth of ChatGPT's usage in the US shows ...,None,"In the US, ChatGPT's usage has grown rapidly, ...",1,1,0,2.469754,ce3c7e45-c861-4ffd-86b4-8f0b32e7d0d3,bc5bb6cf-c252-4d0b-8104-6e09b53cacf0
2,How does the US usage of ChatGPT relate to its...,"Based on the provided context, US ChatGPT usag...",None,"The context indicates that in the US, ChatGPT'...",1,1,0,6.356458,c9cfa223-a8fa-4fab-af57-b4ba99ed5b59,d6a1a586-03b4-40d6-a61f-826b2d891987
3,US ChatGPT use how much in work and non work a...,Based on the provided context:\n\n- ChatGPT us...,None,"In the US, ChatGPT is used extensively, with o...",1,0,0,7.729793,e15b23fa-41a7-4ae7-b2a0-346daa718389,adccbc1b-dcd4-4b5b-a124-66f6be1ef345
4,How does ChatGPT usage vary across different o...,Based on the provided context:\n\nChatGPT usag...,None,Variation in ChatGPT usage by occupation shows...,1,0,0,5.892751,319bdb95-9bce-4cdc-966b-7f340e0c6a51,404829a9-fe0f-4119-8991-859e9effc615
5,How does the message volume comparison between...,"Based on the provided context, the comparison ...",None,The data indicates that from June 2024 to June...,0,0,0,4.667448,500bdfcb-52dd-4b95-a96b-e84fdd94c93e,0ee85406-bb59-42d9-bc02-026090893144
6,how work related message sharing and usage pat...,Based on the provided context:\n\n- Users in h...,None,the context shows that users in highly paid pr...,1,1,0,7.702994,0b2f8451-c293-4c17-9e7e-c2f05782bd65,05c98fca-616b-475a-a339-83897c054c82
7,How do the changes in user behavior and usage ...,"Based on the provided context, from June 2024 ...",None,The data shows that from June 2024 to June 202...,1,1,0,7.383270,cff67315-85e7-45e6-9613-6fc466d5e68e,53fc3b2d-8360-415b-8188-c3e91c95be0b
8,When did ChatGPT launch?,"ChatGPT launched on November 30, 2022.",None,ChatGPT launched in November 2022.,1,1,0,0.962019,9571d574-42ae-44df-b17d-52c2e6709b62,3f9050dc-7dc5-4a63-a1fe-a50972b2c116
9,What SOC means in work stuff?,"Based on the provided context, SOC stands for ...",None,Variation by Occupation Figure 23 shows how Ch...,0,1,0,2.147284,3af5fdd4-02ed-428d-beef-75fe2c5da3b5,b14c373b-dfda-4cb5-ab30-5ebc30d28ec6


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [29]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [30]:
rag_documents = docs

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:



Why would modifying our chunk size modify the performance of our application?

ANSWER: because it affects the semantic matching, smaller chunks = more precise embbeginds, also the LLM is more efficient

In [32]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

### ANSWER: embeddings ARE your RAG system's understanding of meaning. They convert text to vectors - if the vectors poorly represent semantic similarity, retrieval fails even with perfect chunks and prompts.

In [33]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [34]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [35]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [36]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

'Alright, here’s the juicy scoop straight from the digital streets: People are rocking AI not just as a mindless task robot, but as a sharp advisor and research sidekick, seriously leveling up their decision-making game at work. Instead of just automating grunt work, AI is flexing in knowledge-heavy gigs, boosting productivity where the magic lies in quality decisions.\n\nTo break it down—workers tap ChatGPT for solid decision support, digging into info, practical guidance, and writing help. This combo amps up their output, making money moves smarter, faster, and more precise. The AI isn’t just doing jobs; it’s turbocharging humans to win at the economic game by making every call count.\n\nSo the money-making hustle with AI? It’s about augmenting the brainpower, turning ChatGPT into the ultimate workplace co-pilot that elevates knowledge work rather than replacing it. That’s how the future of cash flow’s getting seriously rad. Boom.'

Finally, we can evaluate the new chain on the same test set!

In [ ]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'earnest-shade-27' at:
https://smith.langchain.com/o/9caeceb1-c4f0-4b46-af6b-bedb25ad4d93/datasets/949c36f7-81d7-463b-91d9-a21245747a40/compare?selectedSessions=3140b6a6-18f6-4843-9a1c-de7c571a30fc




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"US ChatGPT use mainly for work or non work, ho...","Alright, strap in for a turbocharged dive into...",None,"In the US, about 70% of ChatGPT consumer queri...",0,0,1,10.533709,3ae81311-9dd5-4446-a1eb-71d9e8d36649,194c71d8-3e9f-4212-aa47-0a1608106f35
1,How does the growth of ChatGPT's usage in the ...,"Alright, strap in for this blazing insight fro...",None,"In the US, ChatGPT's usage has grown rapidly, ...",1,1,1,6.853827,ce3c7e45-c861-4ffd-86b4-8f0b32e7d0d3,2556a423-53fe-4ff9-a53a-1a6c9ddb310b
2,How does the US usage of ChatGPT relate to its...,"Yo, buckle up—here’s the ultra-lit scoop strai...",None,"The context indicates that in the US, ChatGPT'...",1,1,1,7.324036,c9cfa223-a8fa-4fab-af57-b4ba99ed5b59,d8a96987-f83c-4cfa-a2a3-beff383f6537
3,US ChatGPT use how much in work and non work a...,"Yo, let’s dive into the fresh, spicy deets on ...",None,"In the US, ChatGPT is used extensively, with o...",1,0,1,10.753897,e15b23fa-41a7-4ae7-b2a0-346daa718389,1bfc425d-8b24-4a37-b748-490f3fb63375
4,How does ChatGPT usage vary across different o...,"Alright, let's unravel this ChatGPT usage saga...",None,Variation in ChatGPT usage by occupation shows...,1,1,1,6.849890,319bdb95-9bce-4cdc-966b-7f340e0c6a51,9234c22f-1055-450f-9232-d14c00acda5d
5,How does the message volume comparison between...,"Oh, we’re diving deep into the evolution of Ch...",None,The data indicates that from June 2024 to June...,0,0,1,9.110681,500bdfcb-52dd-4b95-a96b-e84fdd94c93e,3f46f770-f39b-465c-93fe-0ff6dbdcf21e
6,how work related message sharing and usage pat...,"Alright, buckle up for a data-driven ride thro...",None,the context shows that users in highly paid pr...,1,1,1,15.286395,0b2f8451-c293-4c17-9e7e-c2f05782bd65,de846c8d-159c-4160-8f96-d7da2121e5f8
7,How do the changes in user behavior and usage ...,"Ah, buckle up — we’re diving deep into the evo...",None,The data shows that from June 2024 to June 202...,1,1,1,12.210953,cff67315-85e7-45e6-9613-6fc466d5e68e,4961d156-9d59-4361-9b51-11017cca56de
8,When did ChatGPT launch?,Buckle up for some AI history that’s straight ...,None,ChatGPT launched in November 2022.,1,1,1,3.385003,9571d574-42ae-44df-b17d-52c2e6709b62,898162cd-a57b-4a0e-8355-2fad3894425a
9,What SOC means in work stuff?,"Alright, buckle up — SOC in the work universe ...",None,Variation by Occupation Figure 23 shows how Ch...,0,1,1,1.866762,3af5fdd4-02ed-428d-beef-75fe2c5da3b5,b5c2f372-2b3c-453e-b8af-51345b264275


: 

#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.